# Importar librerias

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Descomprimir dataset de kaggle

In [ ]:
# Clonar repositorio
!git clone https://github.com/theit0/GreenAI.git

Cloning into 'GreenAI'...
remote: Enumerating objects: 2558, done.
remote: Total 2558 (delta 0), reused 0 (delta 0), pack-reused 2558 (from 2)
Receiving objects: 100% (2558/2558), 123.02 MiB | 15.71 MiB/s, done.


In [ ]:
zip_path = '/content/GreenAI/data/archive.zip'

In [ ]:
import zipfile
import os

extract_path = '/content/descomprimido'

# Crear carpeta si no existe
os.makedirs(extract_path, exist_ok=True)

# Descomprimir
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Descomprimido en:", extract_path)

Descomprimido en: /content/descomprimido


# Configuración del dispositivo

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda


## Preparación de Datos
*    Reutilizamos y adaptamos el código de preprocesamiento de la Semana 2
*    Configuración de rutas y parámetros

In [6]:
DATA_PATH = '/content/descomprimido/Garbage classification/Garbage classification/'
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_EPOCHS = 15
LEARNING_RATE = 0.001

# Clases del dataset

In [7]:
CLASSES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
NUM_CLASSES = len(CLASSES)

print(f"Clases: {CLASSES}")
print(f"Número de clases: {NUM_CLASSES}")

Clases: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Número de clases: 6


In [8]:
class WasteDataset(Dataset):
    """Dataset personalizado para clasificación de residuos"""

    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.classes = CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        # Cargar rutas de imágenes y etiquetas
        self.images = []
        self.labels = []

        for class_name in self.classes:
            class_path = os.path.join(data_path, class_name)
            if os.path.exists(class_path):
                for img_name in os.listdir(class_path):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.images.append(os.path.join(class_path, img_name))
                        self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformaciones para entrenamiento (con data augmentation)

In [9]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Explicación
* transforms.Resize((IMG_SIZE, IMG_SIZE)):
    > Función: Redimensiona todas las imágenes a 224x224 píxeles  
    > Por qué: Las imágenes vienen en diferentes tamaños, pero la red necesita entrada uniforme  
    > Ejemplo: Una imagen de 1920x1080 se convierte a 224x224

* transforms.RandomHorizontalFlip(p=0.5):

    > Función: Voltea horizontalmente la imagen con 50% de probabilidad  
    > Por qué: Data Augmentation - crea variaciones para evitar overfitting  
    > Ejemplo: Una botella de plástico puede aparecer normal o volteada horizontalmente

* transforms.RandomRotation(degrees=15)

    > Función: Rota la imagen aleatoriamente entre -15° y +15°  
    > Por qué: Los residuos en fotos reales pueden estar en cualquier ángulo  
    > Ejemplo: Una lata puede estar inclinada 10° a la izquierda o derecha

* transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)

    > Función: Modifica aleatoriamente:
        - brightness=0.2: Brillo ±20%
        - contrast=0.2: Contraste ±20%
        - saturation=0.2: Saturación ±20%
        - hue=0.1: Matiz ±10%  
    > Por qué: Simula diferentes condiciones de iluminación y cámara  
    > Ejemplo: Una foto puede ser más brillante, más oscura, o con colores más intensos

* transforms.ToTensor()

    > Función: Convierte la imagen PIL a tensor de PyTorch  
    > Cambios:
        - Formato: (H, W, C) → (C, H, W)
        - Valores: [0-255] → [0.0-1.0]  
    > Por qué: PyTorch necesita tensores, no imágenes PIL

* transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    > Función: Normaliza cada canal RGB con media y desviación estándar específicas  
    > Valores: Los de ImageNet (dataset donde se preentrenaron MobileNet y ResNet)
      - Media: [0.485, 0.456, 0.406] (R, G, B)
      - Std: [0.229, 0.224, 0.225] (R, G, B)  
    > Fórmula: pixel_normalizado = (pixel - media) / std  
    > Por qué: Los modelos preentrenados esperan datos en este rango

* Objetivo General:
    Estas transformaciones crean múltiples versiones de cada imagen durante el entrenamiento:
    Imagen Original → [Resize] → [Flip?] → [Rotate] → [ColorJitter] → [ToTensor] → [Normalize]
    Beneficios:

    Previene overfitting: El modelo ve más variaciones
    Mejora generalización: Reconoce residuos en diferentes condiciones
    Aumenta dataset virtual: Una imagen se convierte en miles de variaciones
    Robustez: Funciona con fotos tomadas en diferentes ángulos/iluminación

# Transformaciones para validación (sin augmentation)

In [10]:
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

En la validación se evalúa el modelo tal como esta, no buscando variaciones

# Crear dataset completo y dividir en train/val

In [11]:
full_dataset = WasteDataset(DATA_PATH, transform=train_transform)
dataset_size = len(full_dataset)
print(f"Tamaño total del dataset: {dataset_size}")

Tamaño total del dataset: 2527


# División 80/20 para entrenamiento y validación

In [12]:
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

* se utilizan 80% de los datos para entrenamiento y 20% de los datos para validación siendo el total del dataset de 2527 muestras es un buen porcentaje para seleccionar
* se utiliza manual_seed(42) para siempre obtener la misma generación de datos aleatorios, esto es así para pode controlar si existe un error y el número 42 es un número común utilizado

# Aplicar transformaciones específicas a validación

In [13]:
val_dataset.dataset.transform = val_transform

print(f"Tamaño dataset entrenamiento: {len(train_dataset)}")
print(f"Tamaño dataset validación: {len(val_dataset)}")

Tamaño dataset entrenamiento: 2021
Tamaño dataset validación: 506


# Crear DataLoaders

In [14]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Obtener distribución del dataset de entrenamiento

In [15]:
train_labels = [train_dataset[i][1] for i in range(len(train_dataset))]
class_counts = Counter(train_labels)

print("Distribución de clases en entrenamiento:")
for class_idx, count in sorted(class_counts.items()):
    print(f"{CLASSES[class_idx]}: {count} imágenes")

Distribución de clases en entrenamiento:
cardboard: 326 imágenes
glass: 396 imágenes
metal: 324 imágenes
paper: 478 imágenes
plastic: 393 imágenes
trash: 104 imágenes


# Calcular pesos para el loss balanceado

In [16]:
total_samples = sum(class_counts.values())
class_weights = []
for i in range(NUM_CLASSES):
    weight = total_samples / (NUM_CLASSES * class_counts.get(i, 1))
    class_weights.append(weight)

class_weights_tensor = torch.FloatTensor(class_weights).to(device)
print(f"\nPesos de clases para loss balanceado: {class_weights}")


Pesos de clases para loss balanceado: [1.0332310838445808, 0.8505892255892256, 1.0396090534979423, 0.7046722454672245, 0.8570822731128075, 3.238782051282051]
